**Passage à une fonction: nettoyage et création d'une dataframe contenant le numéro de dossier et l'annotation**

à noter: ca ne marche pas avec les filehtml.ann mais avec les filepdf.ann


In [ ]:
#La fonction pour Thib

def filtrage(source,dest,data):   
    '''fonction pour créer des dataframes contenant la source et destination du lien voulu
    input: les annotation sources, dest et une dataframe 
    output: la dataframe filtrée '''

    data=data[data['class_source']==source]
    data=data[data['class_dest']==dest]

    return(data.drop(['class_source','class_dest'], axis=1))

**On va maintenant créer une fonction qui prend plusieurs dossiers et renvoie une dataframe associée à chaque annotation**

In [5]:
import pandas as pd

In [12]:
def creationDF1(dossiers):
    '''la fonction retourne des dataframes des traitements, examens, pathologies
    input: liste de DPI
    output: dataframes avec une colonne contenant l annotation et une autre le numéro de doss associé
    '''
    
    traitements=pd.DataFrame()
    examens=pd.DataFrame()
    pathologies=pd.DataFrame()
    
    for i in range(len(dossiers)):
        
        df= pd.read_csv(dossiers[i], sep='\t')
        # if len(df.columns) != 3 :
            # print(dossiers[i])
            
        df.columns=['numéro','annotation','detail']
    
        df_ann = df['annotation'].convert_dtypes(convert_string=True)
        df['annotation'] = df_ann.str.split(expand=True)[0]
    
        df.pop('numéro')

        
        df=df.pivot(columns='annotation')

        df=df[('detail')]

        df['Numéro Dossier']= i+1
        
        
        #initialisation des dataframes:
        df_trait=pd.DataFrame()
        df_path=pd.DataFrame()
        df_exam=pd.DataFrame()
    
        
        
        if 'traitement' in df.columns:
            df_trait=df[['Numéro Dossier','traitement']].dropna()
        
        if 'pathologie' in df.columns:
            df_path=df[['Numéro Dossier','pathologie']].dropna()
        
        if 'examen' in df.columns:
            df_exam=df[['Numéro Dossier','examen']].dropna()
        
        traitements= traitements.append(df_trait, ignore_index=True)
        pathologies= pathologies.append(df_path, ignore_index=True)
        examens= examens.append(df_exam, ignore_index=True)
        
        
    #juste pour avoir une visu de la data frame:
    print(traitements)
    print("---------------")
    print(pathologies)
    print("---------------")
    print(examens)
    
    return(traitements, pathologies, examens)
        
        

In [15]:
def creationDF(dossiers, annot):
    '''la fonction retourne la dataframe de l'annotation voulue
    input: liste de DPI, annotation ex 'Traitement'
    output: dataframes avec une colonne contenant l annotation et une autre le numéro de doss associé
    '''
    
    madf=pd.DataFrame()
    
    for i in range(len(dossiers)):
        
        df= pd.read_csv(dossiers[i], sep='\t')
        # if len(df.columns) != 3 :
            # print(dossiers[i])
            
        df.columns=['numéro','annotation','detail']
    
        df_ann = df['annotation'].convert_dtypes(convert_string=True)
        df['annotation'] = df_ann.str.split(expand=True)[0]
    
        df.pop('numéro')

        
        df=df.pivot(columns='annotation')

        df=df[('detail')]

        df['Numéro Dossier']= i+1
        
        
        #initialisation des dataframes:
        df_annot=pd.DataFrame()
    
              
        if annot in df.columns:
            df_annot=df[['Numéro Dossier',annot]].dropna()
        
        madf = madf.append(df_annot, ignore_index=True)
            
    return(madf)
        
        

In [3]:
import os

directory=os.getcwd()
liste_dossiers=[]
for filename in os.listdir(directory):
    if 'filepdf' in filename:
        liste_dossiers.append(filename)

In [21]:
Examens=creationDF(liste_dossiers,'examen')
print(Examens)

annotation  Numéro Dossier                    examen
0                        1       suivi psychiatrique
1                        2               cystoscopie
2                        2           examen clinique
3                        2          contrôle un mois
4                        2               cystoscopie
...                    ...                       ...
1683                   164      analyse histologique
1684                   164           biopsie rectale
1685                   164  exploration endoscopique
1686                   164             échographique
1687                   164       examen endoscopique

[1688 rows x 2 columns]


In [22]:
Examens.examen

0            suivi psychiatrique
1                    cystoscopie
2                examen clinique
3               contrôle un mois
4                    cystoscopie
                  ...           
1683        analyse histologique
1684             biopsie rectale
1685    exploration endoscopique
1686               échographique
1687         examen endoscopique
Name: examen, Length: 1688, dtype: object

** Premiers essais de visu **

In [24]:
len(pd.unique(Examens.examen))

834

In [27]:
#les examens les plus fréquents:
Examens.examen.value_counts( normalize=False, sort=True, ascending=False, bins=None, dropna=True)

examen clinique                                           60
fonction rénale                                           28
échographie                                               26
tension artérielle                                        24
TDM                                                       22
                                                          ..
examen microscopique des fragments biopsiés                1
EOGD                                                       1
ETO                                                        1
contrôle (clinique, biologique et tomodensitométrique)     1
(UIV)                                                      1
Name: examen, Length: 834, dtype: int64

# Machine learning

Quelques essais d'algorithmes de clustering

# Poubelle

In [ ]:
import pandas as pd
def transformation(numeroDossier, dossier):
    df= pd.read_csv(dossier, sep='\t')
    df.columns=['numéro','annotation','detail']
    
    # on transforme les annotations qui sont de type object et précisent
    #  la place dans le dossier du texte annoté en string 
    df_ann=df['annotation'].convert_dtypes(convert_string=True)
    df['annotation']=df_ann.str.split(expand=True)[0]
    
    # on supprime la colonne numéro
    df.pop('numéro')
    
    # on pivote la data frame pour pouvoir trier chaque détail par annotation
    df=df.pivot(columns='annotation')

    # on tej le multiindex détail qui sert à rien
    df=df[('detail')]

    # on ajoute une colonne numéro de doss
    df['Numéro Dossier']= numeroDossier
    
    # on crée des dataframes indépendantes pour les annotations qui nous intéressent:
    # traitement, pathologie et examen
    df_trait=df[['Numéro Dossier','traitement']].dropna()
    df_path=df[['Numéro Dossier','pathologie']].dropna()
    df_exam=df[['Numéro Dossier','examen']].dropna()
    print(df_path)
    print(df_trait)
    
 
# exemple pour 1 fichier:
transformation(numeroDossier=1,dossier='filepdf-5-cas.ann')

transformation(numeroDossier=2,dossier='filepdf-17-cas.ann')

        # dû à un problème de scope des variables de ma fonction transformation, 
        # je dois redéfinir cette même fonction dans celle-ci
        # à l'origine j'avais écrit:
        #transformation(numeroDossier=i+1, dossier=dossiers[i])
        #pathologies.append(df_path)
        #examens.append(df_exam)
        #traitements.append(df_trait)